<a href="https://colab.research.google.com/github/DarshitaBhagat/ImageDetection/blob/main/ImageDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U tensorflow_datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import math
import numpy as np
import matplotlib.pyplot as plt
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


In [3]:
dataset , metadata = tfds.load('fashion_mnist' , as_supervised = True, with_info=True)
train_dataset , test_dataset = dataset['train'] , dataset ['test']

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
class_names=metadata.features['label'].names
print(class_names)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [5]:
#explore the data

num_train_examples= metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print(num_test_examples)
print(num_train_examples)

10000
60000


In [6]:
# preprocess the data

def normalize(images,labels):
  images = tf.cast(images,tf.float32)
  images/=255
  return images , labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

#to keep images in memory
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

In [7]:
# exploring the preprocessed data



In [8]:
# building model

model= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
#compile the model

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [12]:
# training model

BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [13]:
model.fit(train_dataset,epochs=5,steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.7818 - loss: 0.6198
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8644 - loss: 0.3787
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8750 - loss: 0.3431
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8852 - loss: 0.3183
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8910 - loss: 0.2972


In [14]:
#eval acc

test_loss , test_acc = model.evaluate(test_dataset , steps = math.ceil(num_test_examples/BATCH_SIZE))
print(test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8757 - loss: 0.3597
0.8726999759674072


In [25]:
#make predictions
for test_images , test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images) #shape: [batch_size, 28, 28, 1] of test_images then batch size,724 , batch size , 128 , batch size , 10
  #output is a list with 10 values which are probabilities
predictions.shape
predictions[0]
np.argmax(predictions[3])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


np.int64(9)